In [77]:
import pandas_datareader.data as web
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
sns.set_style('darkgrid')
sns.set_palette('viridis')

from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression


In [78]:
def range_slider(fig,buttons = None):
    if not buttons:
        buttons = [
                dict(count=7,
                     label="1w",
                     step="day",
                     stepmode="todate"),
                dict(count=14,
                     label="2w",
                     step="day",
                     stepmode="todate"),
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(step="all")
            ]
    fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(buttons)
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    ))
    
buttons = [
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(count=5,
                     label="5y",
                     step="year",
                     stepmode="backward"),
    
                dict(step="all")
            ]

In [79]:
data = pd.read_csv("Shiller_dat.csv")

In [108]:
data

,Date,P,D,E,B,CPI,Fraction,Rate GS10,Price,Dividend,Price.1,Earnings,Earnings.1,CAPE,TR CAPE,10yr_rolling_avg_dividends,POLY_REG_DIVS
0,1871-01-01,4.440000,0.260000,0.400000,0.000000,12.464061,1871.041667,5.320000,91.333743,5.348372,9.133374e+01,8.228265,8.228265,NaN,NaN,NaN,1.897760
1,1871-02-01,4.500000,0.260000,0.400000,0.000000,12.844641,1871.125000,5.323333,89.825241,5.189903,9.025773e+01,7.984466,8.022910,NaN,NaN,NaN,1.913912
2,1871-03-01,4.610000,0.260000,0.400000,0.000000,13.034972,1871.208333,5.326667,90.677322,5.114122,9.154214e+01,7.867880,7.942919,NaN,NaN,NaN,1.928501
3,1871-04-01,4.740000,0.260000,0.400000,0.000000,12.559226,1871.291667,5.330000,96.766116,5.307846,9.813555e+01,8.165917,8.281481,NaN,NaN,NaN,1.944653
4,1871-05-01,4.860000,0.260000,0.400000,0.000000,12.273812,1871.375000,5.333333,101.523054,5.431275,1.034188e+02,8.355807,8.511837,NaN,NaN,NaN,1.960284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1786,2019-11-01,3104.904500,57.900000,137.280000,89.586325,257.208000,2019.875000,1.810000,3095.078242,57.716761,1.915670e+06,136.845543,84699.296200,29.836868,32.663323,41.215303,30.221157
1787,2019-12-01,3176.749524,58.240000,139.470000,87.985509,256.974000,2019.958333,1.860000,3169.579480,58.108550,1.964779e+06,139.155211,86260.429100,30.331822,33.209288,41.477814,30.236788
1788,2020-01-01,3278.202857,58.686868,131.756667,87.663672,257.971000,2020.041667,1.760000,3258.162907,58.328110,2.022704e+06,130.951226,81295.990600,30.985220,33.923158,41.744587,30.252940
1789,2020-02-01,3277.314211,59.133736,124.043333,87.341836,258.678000,2020.125000,1.500000,3248.377132,58.611614,2.019661e+06,122.948092,76442.321800,30.729689,33.643653,42.015453,30.269092


In [81]:
data.dtypes

Date          float64
P             float64
D             float64
E             float64
B             float64
CPI           float64
Fraction      float64
Rate GS10     float64
Price         float64
Dividend      float64
Price.1       float64
Earnings      float64
Earnings.1    float64
CAPE          float64
TR CAPE       float64
dtype: object

In [82]:
def to_datetime(date):
    ym = str(date).split(".")
    if ym[1] == "1":
        ym[1] = "10"
    ym = list(map(lambda s: int(s), ym))
    date = datetime(ym[0],ym[1],1)
    return date

data.Date = data.Date.apply(lambda d: to_datetime(d))



In [83]:
data.dropna(subset = ['Date', 'P', 'D', 'E', 'B', 'CPI', 'Fraction', 'Rate GS10', 'Price',
       'Dividend', 'Price.1', 'Earnings', 'Earnings.1'],inplace = True)

data.isna().sum()

Date            0
P               0
D               0
E               0
B               0
CPI             0
Fraction        0
Rate GS10       0
Price           0
Dividend        0
Price.1         0
Earnings        0
Earnings.1      0
CAPE          120
TR CAPE       120
dtype: int64

## Approximate Dividend Growth

In [84]:
data["10yr_rolling_avg_dividends"] = data.Dividend.rolling(window=120).mean()

In [85]:
plot = px.line(data,x = 'Date', y = ["Dividend","10yr_rolling_avg_dividends"],
              labels={
                         "value": "Dividend per share ($)"
                     },
              title="Variation in S&P 500 Dividends since 1988")
range_slider(plot,buttons)
plot.show()

In [122]:
training_data = data.loc[1600:]

In [123]:
date = np.array(training_data.Date.astype(int)).reshape(-1,1)

degree=3
polyreg=make_pipeline(PolynomialFeatures(degree),LinearRegression())
polyreg.fit(date,training_data.Dividend)

Pipeline(steps=[('polynomialfeatures', PolynomialFeatures(degree=3)),
                ('linearregression', LinearRegression())])

In [124]:
training_data['POLY_REG_DIVS'] = polyreg.predict(date)

/Users/Ali/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [125]:
plot = px.line(training_data,x = 'Date', y = ["Dividend","10yr_rolling_avg_dividends",'POLY_REG_DIVS'],
              labels={
                         "value": "Dividend per share ($)"
                     },
              title="Variation in S&P 500 Dividends since 1988")
range_slider(plot,buttons)
plot.show()

In [126]:
future_dates = list(map(lambda year : datetime(year,1,1), range(2020,2026)))
future_dates_formatted = np.array(pd.Series(future_dates).astype(int)).reshape(-1,1)

future_payouts = polyreg.predict(future_dates_formatted)

dividend_growth = []

for idx_prev,pay in enumerate(future_payouts[1:]):
    dividend_growth.append(pay/future_payouts[idx_prev] -1)
    
div_growth = pd.DataFrame({"Year":range(2021,2026),"Predicted Dividend Growth":dividend_growth}).set_index('Year').T
div_growth

Year,2021,2022,2023,2024,2025
Predicted Dividend Growth,0.052063,0.051327,0.050734,0.05014,0.049684


From our models predictions, we can see the growth rate seems to be about 5% (reasonable) and decreasing (albeit slowly).

## The DDM Model
The Formula for asset price, as given by the DDM model is the following:
>![\Medium P_1=\frac{D_1}{r-g}](https://latex.codecogs.com/svg.latex?\Large&space;P_1=\frac{D_1}{r-g}) 

Where P is the price, D_1 is the next Dividend payment, r is the COE/ROR, and g is the growth rate. As we know the growth rate, we can also say

>![\Medium P_1=\frac{D_0(1+g)}{r-g}](https://latex.codecogs.com/svg.latex?\Large&space;P_1=\frac{D_0(1+g)}{r-g}) 

Where D_0 is the current divident payment.

#### Obtaining the ROE

In [130]:
start = datetime(2020, 2, 1)
end = datetime(2021, 2, 1) 
SP500 = web.DataReader(['sp500'], 'fred', start, end)
#Drop all Not a number values using drop method.
SP500.dropna(inplace = True)
SP500ROR = (SP500['sp500'].iloc[-1]/ SP500['sp500'].iloc[0])-1
print("YEARLY ROE FOR THE S&P500: {}".format(SP500ROR))

YEARLY ROE FOR THE S&P500: 0.1615736921807862


#### Implementing the DDM
As from our quarterly data, we know the last known dividend sum was $59.20. setting this equal to `D_0` we can now implement the DDM

In [131]:
D_0 = 59.183346
g = np.mean(dividend_growth)

r =SP500ROR
P_1 = D_0*(1+g)/(r-g)
P_1

561.3549908509458

which spits out a value of $517 which is only 20% of the market value. But why is this? 
#### Required Return
16.2% is perhaps too high a return to expect, so we could try settling for a more reasonable 9.5% (J. Sefton)%:

In [132]:
r =0.095
P_1 = D_0*(1+g)/(r-g)
P_1

1406.6632881183007

This gives a price of $1406, which is still 46% lower than the current price. This could be down to the fact that not everyone pays dividends.